### HTTP requests code

In [ ]:
# APIkey: 4nPd9w1xNUFWvYaQzjhLrX8FoPAFqMkb

import datetime

import requests


KEY = "4nPd9w1xNUFWvYaQzjhLrX8FoPAFqMkb"
date = datetime.datetime.now().strftime("%Y%m%d")

BASE_REQUEST = (
        "https://api.nytimes.com/svc/search/v2/articlesearch.json"
)

payload = {
  "api-key": KEY,
  "begin_date": date,
  "end_date": date,
  "q": "Donald Trump"
}

r = requests.get(BASE_REQUEST, params=payload)

if r.status_code == 200:
    print(r.json())
    


### HTML - soup 

In [ ]:
import requests


r = requests.get('https://en.wikipedia.org/wiki/Web_scraping')

if r.status_code == 200:
    print(r.text)

In [ ]:
import requests
#!pip install bs4 
from bs4 import BeautifulSoup


r = requests.get('https://en.wikipedia.org/wiki/Web_scraping')

if r.status_code == 200:
    soup = BeautifulSoup(r.content, "html.parser")
    
    for headline in soup.find_all("span", {"class": "mw-headline"}):
        print(headline.text)
        
        spotligt-tab: <a class="nav-link filters__tab-link js-tab-link active" id="spotlight-tab" data-toggle="tab" href="#spotlight-panel" role="tab" aria-controls="spotlight-panel" aria-selected="true">Spotlights</a>
        company name: <h2 class="ink__title" data-size="lg" data-font-family="unica" data-word-wrap="true" data-font-size="lg">DoorDash</h2>
        company url: <a class="ink" href="https://www.sequoiacap.com/companies/doordash/" style="background-color: #00a071">



### HTTP requests full example

In [ ]:
import random
import re
import sys
import time

import bs4
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup


def delay() -> None:
    time.sleep(random.uniform(15, 30))
    return None


base_url = "https://www.sequoiacap.com/companies/"
content: dict = {
    "name": [],
    "summary": [],
    "url": [],
    "socialmedia": [],
    "description": [],
    "milestones": [],
    "team": [],
    "partners": []
}

delay()
r=requests.get(base_url)

In [ ]:
if r.status_code == 200:
    soup= BeautifulSoup(r.content, "html.parser")
    titles=soup.find_all("h2", {"class": "ink__title"})
    details=soup.find_all("div", {"class": "ink__detail"})
    
    for i in range(len(titles)):
        print(i, 'out of',len(titles))

        socialmedia=[]
        milestones=[]
        team=[]
        partner=[]
        print("############################################################ \nname:",titles[i].text)
        print("summary:",details[i].text)
        full_url="https://www.sequoiacap.com/companies/"+titles[i].text+"/"
        
        rnew = requests.get(full_url)
        detailed_soup = BeautifulSoup(rnew.content, "html.parser")        
        url = detailed_soup.find("a", {"class": "button button--outline-light button--small"})["href"]
        print("url:",url)
        all_socialmedia=detailed_soup.find("section", {"class": "social-sharing__options u-mt-0"}).find_all("a")#.get("href")
        
        for ii in range(len(all_socialmedia)):
            socialmedia.append(all_socialmedia[ii]["href"])
        print("socialmedia:",socialmedia)
        description=detailed_soup.find("div", {"class":"wysiwyg wysiwyg--fs-lg"}).text.strip()
        print("description:",description)
        
        which_list=detailed_soup.find_all("div",{"class":"clist"})
        
        for ii in range(len(which_list)):
            title=which_list[ii].find("h2",{"class":"clist__title"}).text.strip()
            
            if title=="Milestones":
                all_milestones=which_list[ii].find("ul",{"class":"clist__list"}).find_all("li", {"class":"clist__item"})
                for iii in range(len(all_milestones)):
                    stripped_milestones = re.sub(r"[\n\t\s]*", "", all_milestones[iii].text)
                    milestones.append(stripped_milestones)
                print("milestones:",milestones)
        
            elif title=="Team":
                all_team=which_list[ii].find("ul",{"class":"clist__list"}).find_all("li", {"class":"clist__item"})

                for iii in range(len(all_team)):
                    try:
                        stripped_teams = all_team[iii].find("a")["href"]
                    except:
                        stripped_teams = re.sub(r"[\n\t\s]*", "", all_team[iii].text)
                    team.append(stripped_teams)
                print("team",team)

            elif (title=="Partner" or title=="Partners"):
                all_partners=which_list[ii].find("ul",{"class":"clist__list"}).find_all("li", {"class":"clist__item"})

                for iii in range(len(all_partners)):
                    stripped_partners = all_partners[iii].find("a")["href"]
                    partner.append(stripped_partners)
                print("partner:",partner)
            
       #     else:
       #         continue
                
        content["name"].append(titles[i].text)
        content["summary"].append(details[i].text)
        content["url"].append(url)
        content["socialmedia"].append(socialmedia)
        content["description"].append(description)
        content["milestones"].append(milestones)
        content["team"].append(team)
        content["partners"].append(partner)
        delay()           
else:
    print(r.status_code)

    

In [ ]:
df = pd.DataFrame(content)
df = df.replace("", np.nan).fillna(value="NA")
df.to_csv("webscraping_requests_output.csv",
          index=False,
          sep=";",
          encoding="utf-8")

In [ ]:
df_read=pd.read_csv("webscraping_requests_output.csv",sep=';')#.head(5).partner
df_read

### selenium full example

In [1]:
# !conda info
import sys
!{sys.executable} -m pip install selenium
!{sys.executable} -m pip show selenium

  Using cached urllib3-1.26.12-py2.py3-none-any.whl (140 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda 22.9.0 requires ruamel_yaml_conda>=0.11.14, which is not installed.
requests 2.22.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.


Name: selenium
Version: 4.5.0
Summary: 
Home-page: https://www.selenium.dev
Author: 
Author-email: 
License: Apache 2.0
Location: d:\anaconda3\lib\site-packages
Requires: certifi, trio, trio-websocket, urllib3
Required-by: 


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# configure webdriver
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen

# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    # this will disable image loading
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

browser = webdriver.Chrome(r'D:/chromedriver_win32/chromedriver.exe',options=options, chrome_options=chrome_options)
browser.get("https://www.twitch.tv/directory/game/Art")
# wait for page to load
element = WebDriverWait(driver=browser, timeout=5).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-target=directory-first-item]')))
selenium_raw=browser.page_source
browser.close()


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: use options instead of chrome_options


In [9]:
!{sys.executable} -m pip install parsel
from bs4 import BeautifulSoup

from parsel import Selector

sel = Selector(text=selenium_raw)
selenium_content = []
for item in sel.xpath("//div[contains(@class,'tw-tower')]/div[@data-target]"):
    selenium_content.append({
        'title': item.css('h3::text').get(),
        'url': item.css('.tw-link::attr(href)').get(),
        #'username': item.css('.tw-link::text').get(),
        'tags': item.css('.tw-tag ::text').getall(),
        'viewers': ''.join(item.css('.tw-media-card-stat::text').re(r'(\d+)')),
    })

#print(selenium_content)

[{'title': 'BIRTHDAY MARATHON! Begins Oct. 29th at 11:00am ET. !birthdaystream', 'url': '/bobross', 'tags': ['English'], 'viewers': '16'}, {'title': '리노참치 그림방송', 'url': '/rinotuna', 'tags': ['한국어'], 'viewers': '841'}, {'title': '🤖 Cat mecha 🤖 | First mecha commission | Sketching a new waifu later!', 'url': '/tofusenshi', 'tags': ['czech', 'artcommission'], 'viewers': '229'}, {'title': 'Working on covers...| !store !links', 'url': '/sozomaika', 'tags': ['English'], 'viewers': '234'}, {'title': '[ Vtuber ITA ] ~ SUBATHON di Compleanno! Day 7 AAAAAAH ! ~ !throne per farmi un regalo di compleanno !', 'url': '/cappuccino_chan', 'tags': ['italiano', 'art', 'illustration'], 'viewers': '64'}, {'title': '✨🎨 your daily dose of motivation 👩\u200d🎨✨', 'url': '/vivisartservice', 'tags': ['English', 'art', 'positive'], 'viewers': '93'}, {'title': "art strim٩( ''ω'' )و", 'url': '/ototaaan', 'tags': ['chill', '日本語', 'illustrator'], 'viewers': '32'}, {'title': 'やほ', 'url': '/kingu_ch', 'tags': ['Comfy'

In [10]:
import pandas as pd
import numpy as np

df_selenium = pd.DataFrame(selenium_content)
df_selenium = df_selenium.replace("", np.nan).fillna(value="NA")
df_selenium.to_csv("webscraping_selenium_output.csv",
          index=False,
          sep=";",
          encoding="utf-8")
df_read=pd.read_csv("webscraping_selenium_output.csv",sep=';')#.head(5).partner
df_read

,title,url,tags,viewers
0,BIRTHDAY MARATHON! Begins Oct. 29th at 11:00am...,/bobross,['English'],16
1,리노참치 그림방송,/rinotuna,['한국어'],841
2,🤖 Cat mecha 🤖 | First mecha commission | Sketc...,/tofusenshi,"['czech', 'artcommission']",229
3,Working on covers...| !store !links,/sozomaika,['English'],234
4,[ Vtuber ITA ] ~ SUBATHON di Compleanno! Day 7...,/cappuccino_chan,"['italiano', 'art', 'illustration']",64
5,✨🎨 your daily dose of motivation 👩‍🎨✨,/vivisartservice,"['English', 'art', 'positive']",93
6,art strim٩( ''ω'' )و,/ototaaan,"['chill', '日本語', 'illustrator']",32
7,やほ,/kingu_ch,"['Comfy', 'English', '日本語']",59
8,[18+] Traditional art thursdays! Trying waterc...,/redbeanporridge,"['traditionalArt', 'Productivity']",95
9,frogs are back and trying their best~ // !shop...,/artbyzvesta,"['DigitalArt', 'lgbtqia', 'Cosy']",126


### playwright example: has error

In [11]:
!{sys.executable} -m pip install playwright
!playwright install
!{sys.executable} -m pip install ipython nest_asyncio

Node.js is only supported on Windows 8.1, Windows Server 2012 R2, or higher.
Setting the NODE_SKIP_PLATFORM_CHECK environment variable to 1 skips this
check, but Node.js might not execute correctly. Any issues encountered on
unsupported platforms will not be fixed.


In [14]:

## error : module 'attr' has no attribute 'frozen'

!{sys.executable} -m from playwright.sync_api import sync_playwright

with sync_playwright() as pw:
    browser = pw.chromium.launch(headless=False)
    context = browser.new_context(viewport={"width": 1920, "height": 1080})
    page = context.new_page()

    page.goto("https://twitch.tv/directory/game/Art")  # go to url
    page.wait_for_selector("div[data-target=directory-first-item]")  # wait for content to load
    print(page.content())
    selenium_content=page.content()
    
parsed = []
for item in soup.select(".tw-tower div[data-target]"):
    parsed.append({
        'title': item.select_one('h3').text,
        'url': item.select_one('.tw-link::attr(href)').attrs.get("href"),
        #'username': item.select_one('.tw-link').text,
        'tags': [tag.text for tag in item.select('.tw-tag')],
        'viewers': item.select_one('.tw-media-card-stat').text,
    })


D:\Anaconda3\python.exe: No module named from


NameError: name 'sync_playwright' is not defined